# ALL installs 

In [45]:
# ! pip install requests"
# ! pip install kafka-python
# ! pip install pyspark
# !pip  install -r requirements.txt

In [46]:
!pip freeze > requirements.txt


# ALL imports

In [47]:
from allsecrets import *
from titles import *
from coordinates import *
import json,requests
from json import dumps,loads 
#dumps converts python object to json string
#loads converts json string to python object
from kafka import KafkaProducer, KafkaConsumer




In [48]:
from pyspark.sql import SparkSession , functions as F

from pyspark.sql.functions import udf,col,countDistinct,date_format,row_number

from pyspark.sql.types import FloatType

from pyspark.sql.window import Window 

spark = SparkSession.builder.appName('OMDB')\
    .getOrCreate()



## Json print function

In [49]:
def json_print(json_object):
    text = json.dumps(json_object, sort_keys=True, indent=4) # sort_keys=True means sort the keys in alphabetical order
    # indent 4 means 4 spaces for each indentation
    print(text)

## OMDB dataset

In [50]:
# # http://www.omdbapi.com/?i=tt3896198&apikey=******* # api key for    OMBD API

# parameters = {'t':'top gun maverick', "apikey": ombd_api} 
# OMDB_response = requests.get("http://www.omdbapi.com/", params=parameters)
# # print url
# print(OMDB_response.url)
# print(OMDB_response.status_code)
# json_print(OMDB_response.json())


In [51]:
# # http://www.omdbapi.com/?i=tt3896198&apikey=******* # api key for    OMBD API

# for title in titles:
#     parameters = {'t': title, "apikey": ombd_api}
#     OMDB_response = requests.get("http://www.omdbapi.com/", params=parameters)
#     print(OMDB_response.status_code)
#     json_print(OMDB_response.json())

## Openweather 5 days 3 hour forcast

In [83]:

parameters = {'lat': '27.7167', "lon": '85.3667', "appid": open_weather_api}
sample_response = requests.get("http://api.openweathermap.org/data/2.5/forecast", params=parameters)


weather_rdd= spark.sparkContext.parallelize([json.dumps(sample_response.json()) ]) # convert to rdd

#convert to spark dataframe
weather_df = spark.read.json(weather_rdd) # convert to spark dataframe

schema_weather = weather_df.schema # get the schema of the dataframe

#  save  schema_weather  to .json file
with open('schema_weather.json', 'w') as f:
    json.dump(schema_weather.jsonValue(), f, indent=4)

#  load  schema_weather  from .json file
with open('schema_weather.json', 'r') as f:
    schema_weather1 = F.StructType.fromJson(json.load(f)) 

# schema_weather1


# printscchema of the dataframe
weather_df.printSchema()





root
 |-- city: struct (nullable = true)
 |    |-- coord: struct (nullable = true)
 |    |    |-- lat: double (nullable = true)
 |    |    |-- lon: double (nullable = true)
 |    |-- country: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- population: long (nullable = true)
 |    |-- sunrise: long (nullable = true)
 |    |-- sunset: long (nullable = true)
 |    |-- timezone: long (nullable = true)
 |-- cnt: long (nullable = true)
 |-- cod: string (nullable = true)
 |-- list: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- clouds: struct (nullable = true)
 |    |    |    |-- all: long (nullable = true)
 |    |    |-- dt: long (nullable = true)
 |    |    |-- dt_txt: string (nullable = true)
 |    |    |-- main: struct (nullable = true)
 |    |    |    |-- feels_like: double (nullable = true)
 |    |    |    |-- grnd_level: long (nullable = true)
 |    |    |    |-- humidity: long (nul

In [52]:
#api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={API key}

for lat ,lon in lat_lon.items():
    parameters = {'lat': lat, "lon": lon, "appid": open_weather_api}
    weather_response = requests.get("http://api.openweathermap.org/data/2.5/forecast", params=parameters)
    print (weather_response.url)
    print(weather_response.status_code)
    # json_print(weather_response.json())



http://api.openweathermap.org/data/2.5/forecast?lat=27.7167&lon=85.3667&appid=8cf7d4ce0b67a86cf3b125bc5c552a6d
200
http://api.openweathermap.org/data/2.5/forecast?lat=28.2097&lon=83.985&appid=8cf7d4ce0b67a86cf3b125bc5c552a6d
200


# Test produces and consumer

## Test producer

In [ ]:
# zookeeper and kafka broker should be running in the background
producer = KafkaProducer(bootstrap_servers=['localhost:9092'], value_serializer=lambda x: dumps(x).encode('utf-8'))
#value_serializer=lambda x: dumps(x).encode('utf-8') is used to convert the data into json format

# send the Weather_response to the topic named Weather
producer.send('WEATHER', value=Weather_response.json())

# flush the messages to the topic
producer.flush() # flush() is a blocking function which ensures all the messages are sent to the topic


## Test consumer

In [76]:
# create consumer
Consumer = KafkaConsumer('weather', bootstrap_servers=['localhost:9092'], auto_offset_reset='earliest',
                         enable_auto_commit=True, value_deserializer=lambda x: loads(x.decode('utf-8'), key_deserializer=lambda x: loads(x.decode('utf-8'))))

In [66]:
# close consumer to free up resources after reading the messages
Consumer.close()

In [77]:
Consumer.topics()

{'_confluent-command',
 '_confluent-telemetry-metrics',
 '_confluent_balancer_api_state',
 '_confluent_balancer_broker_samples',
 '_confluent_balancer_partition_samples',
 'weather'}

In [62]:
Consumer.subscribe(['weather'])
Consumer.subscription()


{'weather'}

In [25]:
# create topic
! kafka-topics --bootstrap-server localhost:9092 --create --topic weather --replication-factor 1 --partitions 1



Created topic weather.


In [63]:
# delete topic
! kafka-topics --bootstrap-server localhost:9092 --delete --topic weather